In [49]:
pip install pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.8 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 89.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 36.7 MB/s eta 0:00:01
     |████████████████████████████████| 249 kB 99.5 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 89.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

class SalesAnalyzer:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.preprocess()

    def preprocess(self):
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        self.df['Quantity'] = pd.to_numeric(self.df['Quantity'], errors='coerce')
        self.df['Unit Price'] = self.df['Unit Price'].replace({',': ''}, regex=True).astype(float)
        self.df['Cost Price'] = self.df['Cost Price'].replace({',': ''}, regex=True).astype(float)
        self.df['Credit Days'] = pd.to_numeric(self.df['Credit Days'], errors='coerce')

        self.df['Total Revenue'] = self.df['Quantity'] * self.df['Unit Price']
        self.df['Total Cost'] = self.df['Quantity'] * self.df['Cost Price']
        self.df['Profit'] = self.df['Total Revenue'] - self.df['Total Cost']
        self.df['Due Date'] = self.df['Date'] + pd.to_timedelta(self.df['Credit Days'], unit='D')
        self.df['Month'] = self.df['Date'].dt.to_period('M')

    # BASIC METRICS
    def get_summary(self):
        return {
            'Total Revenue': self.df['Total Revenue'].sum(),
            'Total Profit': self.df['Profit'].sum(),
            'Gross Margin %': (self.df['Profit'].sum() / self.df['Total Revenue'].sum()) * 100,
            'Average Order Value': self.df['Total Revenue'].sum() / len(self.df),
            'Unique Customers': self.df['Customer Name'].nunique(),
            'Unique Products': self.df['Product'].nunique(),
        }

    # PERFORMANCE & PROFITABILITY
    def best_selling_products(self, n=5):
        return self.df.groupby('Product')['Quantity'].sum().sort_values(ascending=False).head(n)

    def least_profitable_products(self, n=5):
        return self.df.groupby('Product')['Profit'].sum().sort_values().head(n)

    def profit_per_customer(self):
        return self.df.groupby('Customer Name')['Profit'].sum().sort_values(ascending=False)

    def sales_growth_rate(self):
        monthly = self.monthly_revenue()
        return monthly.pct_change().fillna(0)

    # RISK & OPERATIONS
    def aging_receivables(self):
        today = pd.Timestamp.now()
        self.df['Days Overdue'] = (today - self.df['Due Date']).dt.days
        return self.df[self.df['Days Overdue'] > 0][['Customer Name', 'Days Overdue']].groupby('Customer Name').mean()

    def credit_risk_score(self):
        return self.df.groupby('Customer Name')['Credit Days'].mean().sort_values(ascending=False)

    # ADVANCED CUSTOMER INSIGHTS
    def customer_lifetime_value(self):
        revenue = self.df.groupby('Customer Name')['Total Revenue'].sum()
        orders = self.df.groupby('Customer Name').size()
        return (revenue / orders).sort_values(ascending=False)  # avg value per order = CLV approx.

    def repeat_purchase_rate(self):
        order_counts = self.df.groupby('Customer Name').size()
        repeat_customers = order_counts[order_counts > 1].count()
        total_customers = order_counts.count()
        return repeat_customers / total_customers if total_customers > 0 else 0

    def top_regions_by_growth(self):
        monthly = self.df.groupby(['Month', 'Location'])['Total Revenue'].sum().reset_index()
        monthly['Revenue Growth'] = monthly.groupby('Location')['Total Revenue'].pct_change()
        return monthly.sort_values(by='Revenue Growth', ascending=False).dropna()

    # EXISTING ANALYTICS
    def top_customers(self, n=5):
        return self.df.groupby('Customer Name')['Total Revenue'].sum().sort_values(ascending=False).head(n)

    def product_performance(self):
        return self.df.groupby('Product')[['Total Revenue', 'Profit']].sum().sort_values(by='Total Revenue', ascending=False)

    def monthly_revenue(self):
        return self.df.groupby('Month')['Total Revenue'].sum()

    def location_performance(self):
        return self.df.groupby('Location')['Total Revenue'].sum().sort_values(ascending=False)

    def average_credit_days(self):
        return self.df.groupby('Customer Name')['Credit Days'].mean().sort_values(ascending=False)

    def raw(self):
        return self.df

    def to_json_summary(self):
        return {
            'summary': self.get_summary(),
            'top_customers': self.top_customers(5).to_dict(),
            'best_selling_products': self.best_selling_products(5).to_dict(),
            'least_profitable_products': self.least_profitable_products(5).to_dict(),
            'profit_per_customer': self.profit_per_customer().to_dict(),
            'sales_growth_rate': self.sales_growth_rate().round(2).to_dict(),
            'aging_receivables': self.aging_receivables().round(1).to_dict(),
            'credit_risk_score': self.credit_risk_score().round(1).to_dict(),
            'customer_lifetime_value': self.customer_lifetime_value().round(2).to_dict(),
            'repeat_purchase_rate': round(self.repeat_purchase_rate(), 2),
            'top_regions_by_growth': self.top_regions_by_growth().to_dict(orient='records'),
            'product_performance': self.product_performance().round(2).to_dict(orient='index'),
            'monthly_revenue': self.monthly_revenue().round(2).to_dict(),
            'location_performance': self.location_performance().round(2).to_dict(),
            'average_credit_days': self.average_credit_days().round(1).to_dict()
        }


    # VISUALIZATIONS
    def plot_monthly_revenue(self):
        revenue = self.monthly_revenue()
        revenue.plot(kind='line', marker='o', title='Monthly Revenue')
        plt.xlabel('Month')
        plt.ylabel('Revenue')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    def plot_top_customers(self, n=5):
        top = self.top_customers(n)
        top.plot(kind='bar', title='Top Customers by Revenue')
        plt.ylabel('Revenue')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    def plot_product_performance(self):
        perf = self.product_performance()
        perf.plot(kind='bar', figsize=(10,5), title='Product Performance')
        plt.ylabel('Amount')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Replace this with your actual Sheet ID and GID
sheet_id = "1Tm_zp9TkYLLs7ZIUFGA-qDImWU1o-IBbGiKdS8QktNc"
gid = "9859409"

# Construct the CSV export URL
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"

# Read into DataFrame
df = pd.read_csv(csv_url)

# Display the data
df.head()

,Date,Customer Name,Product,Location,Quantity,Cost Price,Unit Price,Credit Days
0,2025-04-01,JaySteel,Iron Ore,Raipur,20,6109,4312,42
1,2025-04-01,Zenith Power,Iron Ore,Raipur,24,5364,4571,35
2,2025-04-02,Bharat Trading,Manganese Ore,Kolkata,17,3614,3819,27
3,2025-04-02,Greenline Corp,Indonesian Coal,Surat,22,7247,7857,35
4,2025-04-02,AlphaEnergy,Manganese Ore,Kolkata,22,3676,3879,32


In [3]:
# assuming you've already loaded the Google Sheet into a DataFrame called df

analyzer = SalesAnalyzer(df)

print("📊 Summary:")
print(analyzer.get_summary())

print("\n💰 Top Customers:")
print(analyzer.top_customers())

print("\n📦 Product Performance:")
print(analyzer.product_performance())

print("\n📆 Monthly Revenue:")
print(analyzer.monthly_revenue())

print("\n📍 Location Performance:")
print(analyzer.location_performance())

print("\n⏳ Average Credit Days by Customer:")
print(analyzer.average_credit_days())


📊 Summary:
{'Total Revenue': np.float64(28573153.0), 'Total Profit': np.float64(49859.0), 'Gross Margin %': np.float64(0.1744959682958335), 'Average Order Value': np.float64(98528.11379310345), 'Unique Customers': 15, 'Unique Products': 4}

💰 Top Customers:
Customer Name
Bharat Trading    4301327.0
AlphaEnergy       3829571.0
Zenith Power      3743894.0
JaySteel          3632025.0
Om Metals         3116151.0
Name: Total Revenue, dtype: float64

📦 Product Performance:
                 Total Revenue     Profit
Product                                  
Indonesian Coal     12592709.0  1499223.0
Iron Ore             6843797.0 -1951672.0
Manganese Ore        5445988.0   239075.0
Pet Coke             3690659.0   263233.0

📆 Monthly Revenue:
Month
2025-04    6821672.0
2025-05    5142191.0
2025-06    6007889.0
2025-07    5391552.0
2025-08    5209849.0
Freq: M, Name: Total Revenue, dtype: float64

📍 Location Performance:
Location
Surat      12592709.0
Raipur      6843797.0
Kolkata     5445988.0


In [4]:
analyzer.least_profitable_products()

Product
Iron Ore          -1951672.0
Manganese Ore       239075.0
Pet Coke            263233.0
Indonesian Coal    1499223.0
Name: Profit, dtype: float64

In [5]:
analyzer.monthly_revenue()

Month
2025-04    6821672.0
2025-05    5142191.0
2025-06    6007889.0
2025-07    5391552.0
2025-08    5209849.0
Freq: M, Name: Total Revenue, dtype: float64